# 🚀 Building a Startup Advisor System: Step-by-Step Tutorial

In this tutorial, we'll build a complete startup advisor system using AI agents. We'll start simple and progressively add more capabilities:

**What You'll Learn:**
- How to create basic AI agents with specific roles
- How to connect agents to external data sources (web search)
- How to coordinate multiple agents working together
- How to create specialized agent teams
- How to generate and save professional reports

**System Architecture:**
```
Basic Agent → Research Agent → Agent Swarm → Specialized Teams → Complete System
```

Let's get started! 🎯


## 🛠️ Setup and Configuration

First, let's import our libraries and set up the foundation for our agents.


In [1]:
# Core imports for building our agent system
import os
import textwrap
from dotenv import load_dotenv

# Strands framework for AI agents
from strands import Agent
from strands.models import BedrockModel

# Load environment variables (API keys)
load_dotenv()

print("✅ Libraries imported successfully!")
print("🔑 Environment variables loaded")


✅ Libraries imported successfully!
🔑 Environment variables loaded


In [5]:
# Configure our AI model
# We're using Amazon's Nova Micro for fast, cost-effective responses
model = BedrockModel(
    model_id="us.amazon.nova-micro-v1:0",
    region_name="us-west-2"
)

print("🧠 AI model configured: Amazon Nova Micro")
print("🌍 Region: us-west-2")


🧠 AI model configured: Amazon Nova Micro
🌍 Region: us-west-2


In [3]:
# Configure file operations to avoid prompts
os.environ["STRANDS_AUTO_APPROVE_FILE_OPERATIONS"] = "true"
os.environ["STRANDS_FORCE_FILE_WRITE"] = "true"

# Utility function for consistent, readable output
DISPLAY_WIDTH = 120

def print_response(text, title=None, width=DISPLAY_WIDTH):
    """Display agent responses in a clean, formatted way"""
    if title:
        print(f"\n🤖 {title}")
        print("─" * width)
    
    # Wrap text to fit our display width
    wrapped = textwrap.fill(str(text), width=width)
    print(wrapped)
    
    if title:
        print("─" * width)

def print_step(step_num, title):
    """Print step headers consistently"""
    print(f"\n{'='*60}")
    print(f"🎯 STEP {step_num}: {title}")
    print(f"{'='*60}")

print("📋 Utility functions ready!")
print("🔧 File operations configured for auto-approval")


📋 Utility functions ready!


---

## 🎯 Step 1: Basic Startup Advisor

Let's start with the simplest possible agent - one that can give basic startup advice without any external tools. This demonstrates how to:
- Create an agent with a specific role
- Define a clear system prompt
- Get basic responses


In [6]:
print_step(1, "Basic Startup Advisor")

# Create our first agent with a specific role and personality
basic_advisor = Agent(
    system_prompt="""You are an experienced startup advisor with 10+ years helping entrepreneurs. 
    Provide practical, actionable advice that's encouraging but realistic. 
    Keep responses concise and focus on the most important points.""",
    model=model
)

# Test our basic advisor
startup_idea = "A mobile app that helps people find and share local hiking trails"

print(f"💭 Startup Idea: {startup_idea}")
print("\n🤔 Asking for initial advice...")

response = basic_advisor(f"Give me your initial thoughts on this startup idea: {startup_idea}");

print_response(response, "Basic Advisor Response")

print("\n✅ Step 1 Complete: Our basic advisor can provide general startup guidance!")



🎯 STEP 1: Basic Startup Advisor
💭 Startup Idea: A mobile app that helps people find and share local hiking trails

🤔 Asking for initial advice...
Your idea for a mobile app to help people find and share local hiking trails is promising and has a lot of potential. Here are some key points to consider:

1. **Market Demand**: There's a growing interest in outdoor activities, especially hiking. Your app could tap into this trend by providing a platform for enthusiasts to discover new trails and share their experiences.

2. **Unique Selling Proposition (USP)**: To stand out, focus on what makes your app unique. This could be detailed trail maps, user-generated content, safety features, or integration with fitness tracking.

3. **User Experience**: Ensure the app is user-friendly. Simple navigation, clear trail descriptions, and easy-to-use features like filtering and sorting will be crucial.

4. **Community Building**: Foster a community around your app. Encourage users to share photos, re

---

## 🎯 Step 2: Research-Powered Agent

Now let's upgrade our agent with web search capabilities using **MCP (Model Context Protocol)**. This allows our agent to:
- Search the web for current market data
- Find competitor information
- Get real-time industry insights


In [8]:
print_step(2, "Research-Powered Agent with Web Search")

# Import MCP libraries for connecting to external services
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Check if we have the required API key
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if PERPLEXITY_API_KEY:
    print("✅ Perplexity API key found - research capabilities enabled!")
    
    # Create connection to Perplexity search service via Docker
    search_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="docker",
                args=[
                    "run", "-i", "--rm", 
                    "-e", "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask"
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY}
            )
        )
    )
    
else:
    print("⚠️ No Perplexity API key found")
    print("💡 Add PERPLEXITY_API_KEY to your .env file to enable web search")
    search_client = None



🎯 STEP 2: Research-Powered Agent with Web Search
✅ Perplexity API key found - research capabilities enabled!


In [ ]:
# Function to test our research agent
def test_research_agent():
    """Test the research agent with web search capabilities"""
    
    if not search_client:
        print("❌ Research agent not available - need Perplexity API key")
        return
    
    with search_client:
        # Get available search tools
        search_tools = search_client.list_tools_sync()
        
        print(f"🛠️ Available tools: {[tool.tool_name for tool in search_tools]}")
        
        # Create research agent with search capabilities
        research_agent = Agent(
            system_prompt="""You are a market researcher who finds current, factual information.
            Use web search to gather data about markets, competitors, and trends.
            Provide specific data points and cite your sources. Keep responses focused and informative.""",
            model=model,
            tools=search_tools  # This gives the agent search capabilities!
        )
        
        # Test the research capabilities
        query = "hiking app market size and main competitors 2024"
        print(f"🔍 Research Query: {query}")
        
        result = research_agent(query);
        
        print_response(result, "Research Agent Results")
        
        return result

# Test our enhanced agent
research_result = test_research_agent()

if research_result:
    print("\n✅ Step 2 Complete: Our agent can now search the web for real-time market data!")
else:
    print("\n⏭️ Step 2 Skipped: Set up Perplexity API key to test web search capabilities")


In [ ]:
print_step(3, "Multi-Agent Collaboration with Swarm")

# Import the swarm tool for agent coordination
from strands_tools import swarm

print("🐝 Swarm tool imported - ready for multi-agent coordination!")

def test_swarm_collaboration():
    """Test coordinating multiple agents with swarm"""
    
    if not search_client:
        print("⚠️ Swarm demo requires web search - using basic version")
        
        # Create a coordinator without search tools
        team_coordinator = Agent(
            system_prompt="""You are a Team Lead who coordinates research specialists.
            Use the swarm tool to create specialist agents for different research areas.
            Assign clear, specific tasks to each agent you create.""",
            model=model,
            tools=[swarm]  # Only swarm tool for coordination
        )
        
        tools_available = "basic reasoning only"
    else:
        # Create coordinator with both search and swarm tools
        with search_client:
            search_tools = search_client.list_tools_sync()
            
            team_coordinator = Agent(
                system_prompt="""You are a Research Team Lead who coordinates market research.
                Use the swarm tool to create specialist agents for different research areas.
                Each specialist agent inherits your search tools and can research their assigned topic.
                Coordinate the team to provide comprehensive market analysis.""",
                model=model,
                tools=search_tools + [swarm]  # Search tools + coordination
            )
            
            tools_available = f"web search + swarm coordination"
    
    print(f"👥 Team Coordinator created with: {tools_available}")
    
    # Give the coordinator a complex task requiring multiple specialists
    complex_task = """
    Research the outdoor recreation app market for a hiking trail app startup.
    
    Create a swarm of 2 specialist agents to research:
    1. Market size, growth trends, and user demographics
    2. Key competitors, their features, and market positioning
    
    Coordinate the research and provide a comprehensive summary.
    """
    
    print("📋 Assigning complex research task to team coordinator...")
    
    team_result = team_coordinator(complex_task);
    
    print_response(team_result, "Team Coordination Results")
    
    return team_result

# Test multi-agent collaboration
swarm_result = test_swarm_collaboration()

print("\n✅ Step 3 Complete: Multiple agents can now work together as a coordinated team!")


In [7]:
print_step(4, "Specialized Agent Roles - Market Analyst & Strategic Advisor")

def create_specialist_team(available_tools=None):
    """Create our specialized agent team"""
    
    # Default to just swarm if no tools provided
    if available_tools is None:
        available_tools = [swarm]
    
    # Market Analyst - specializes in data gathering and analysis
    market_analyst = Agent(
        system_prompt="""You are a Senior Market Analyst at a top consulting firm.
        Your expertise is gathering market data, analyzing competitors, and identifying trends.
        You provide detailed, data-driven insights with specific numbers and facts.
        Always support your analysis with concrete data points.""",
        model=model,
        tools=available_tools
    )
    
    # Strategic Advisor - specializes in business strategy and recommendations
    strategic_advisor = Agent(
        system_prompt="""You are a Strategic Business Advisor with 15+ years of experience.
        You excel at translating market research into actionable business strategies.
        Focus on practical recommendations for positioning, pricing, marketing, and growth.
        Provide clear, implementable advice that startups can act on immediately.""",
        model=model
    )
    
    print("👥 Specialist team created:")
    print("   📊 Market Analyst - Data gathering and competitive analysis")
    print("   💡 Strategic Advisor - Business strategy and recommendations")
    
    return market_analyst, strategic_advisor

# Create our specialist team with proper session management
if search_client:
    # Get tools within the active session
    with search_client:
        search_tools = search_client.list_tools_sync()
        analyst, strategist = create_specialist_team([swarm] + search_tools)
        print(f"🛠️ Market Analyst equipped with: {len(search_tools)} search tools + swarm")



🎯 STEP 4: Specialized Agent Roles - Market Analyst & Strategic Advisor


NameError: name 'search_client' is not defined

In [ ]:
def run_specialist_workflow(startup_idea):
    """Run a complete workflow using our specialist agents"""
    
    print(f"🎯 Analyzing startup: {startup_idea}")
    
    # PHASE 1: Market Analysis
    print("\n📊 PHASE 1: Market Analysis")
    print("-" * 50)
    
    research_task = f"""
    Analyze the market for this startup idea: {startup_idea}
    
    Research and provide:
    1. Market size and growth potential
    2. Key competitors and their market share
    3. Target customer segments and their needs
    4. Market trends and opportunities
    
    Use available tools to gather current market data.
    """
    
    # Execute research within proper session context
    if search_client:
        with search_client:
            market_research = analyst(research_task);
    else:
        market_research = analyst(research_task);
    
    print("✅ Market analysis complete!")
    
    # PHASE 2: Strategy Development
    print("\n💡 PHASE 2: Strategy Development")
    print("-" * 50)
    
    strategy_task = f"""
    Based on this market research, develop a business strategy for: {startup_idea}
    
    Market Analysis:
    {market_research}
    
    Provide strategic recommendations for:
    1. Value proposition and positioning
    2. Target customer segments and personas
    3. Competitive differentiation
    4. Go-to-market strategy
    5. Revenue model and pricing
    6. Key success metrics to track
    """
    
    business_strategy = strategist(strategy_task);
    
    print("✅ Strategy development complete!")
    
    return market_research, business_strategy

# Test our specialist workflow
test_startup = "A mobile app that uses AR to help people learn cooking skills through interactive recipes"

print("🚀 Running specialist workflow...")
research_results, strategy_results = run_specialist_workflow(test_startup)

# Display results
print_response(str(research_results)[:500] + "...", "Market Research Summary (Truncated)")
print_response(str(strategy_results)[:500] + "...", "Strategic Recommendations Summary (Truncated)")

print("\n✅ Step 4 Complete: Specialized agents provide focused expertise in their domains!")


In [ ]:
print_step(5, "Content Creation and File Operations")

# Import file operation tools
from strands_tools import file_write, editor
from strands.tools import tool

# Method 1: Try to configure the original file_write tool
import sys
from io import StringIO

def configure_file_write_auto_approve():
    """Configure file_write to auto-approve operations"""
    # Try to patch the file_write tool to auto-approve
    if hasattr(file_write, '_original_call'):
        return  # Already patched
    
    original_call = file_write._tool_function if hasattr(file_write, '_tool_function') else None
    
    def auto_approve_wrapper(*args, **kwargs):
        # Redirect stdin to automatically respond 'y' to prompts
        old_stdin = sys.stdin
        sys.stdin = StringIO('y\n')
        try:
            if original_call:
                result = original_call(*args, **kwargs)
            else:
                result = file_write(*args, **kwargs)
            return result
        finally:
            sys.stdin = old_stdin
    
    if original_call:
        file_write._tool_function = auto_approve_wrapper
        file_write._original_call = original_call

# Apply the configuration
configure_file_write_auto_approve()

# Method 2: Create a custom file write tool that doesn't prompt
@tool
def auto_file_write(filename: str, content: str) -> str:
    """Write content to a file automatically without prompts"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ Successfully saved file: {filename}"
    except Exception as e:
        return f"❌ Error saving file {filename}: {str(e)}"

print("🔧 File write tools configured for automatic operation")

def create_content_team():
    """Create agents that can generate content and save files"""
    
    # Content Writer - creates engaging marketing materials
    content_writer = Agent(
        system_prompt="""You are a Creative Marketing Writer specializing in startup marketing.
        You excel at creating compelling copy, campaign ideas, and promotional content.
        Transform business strategies into engaging narratives that resonate with target audiences.
        Focus on benefits, emotional appeal, and clear calls to action.""",
        model=model
    )
    
    # Report Writer - creates professional documentation
    report_writer = Agent(
        system_prompt="""You are a Professional Report Writer who creates executive-level documents.
        Take research and strategy outputs and format them into clean, professional reports.
        Use proper markdown structure, headings, and formatting for easy reading.
        Always use the auto_file_write tool to save reports automatically without prompts.""", 
        model=model,
        tools=[auto_file_write, editor]  # Custom auto-save tool + editor
    )
    
    print("✍️ Content team created:")
    print("   📝 Content Writer - Marketing materials and campaigns")
    print("   📄 Report Writer - Professional documentation")
    
    return content_writer, report_writer

# Create our content team
writer, reporter = create_content_team()


In [ ]:
def save_to_file(content, filename):
    """Save content to file without any prompts"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(str(content))
        print(f"✅ Report saved to: {filename}")
        return True
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        return False

def test_content_creation():
    """Test content creation and file operations"""
    
    # Sample business strategy (normally this comes from previous steps)
    sample_strategy = """
    Business Strategy for AR Cooking Learning App:
    
    • Target Market: Millennials and Gen Z who want to learn cooking (ages 22-40)
    • Value Proposition: "Learn cooking like having a chef in your kitchen"
    • Key Features: AR recipe overlays, real-time guidance, ingredient recognition
    • Revenue Model: Freemium with premium recipes and advanced features
    • Go-to-Market: Social media, cooking influencers, kitchen appliance partnerships
    """
    
    print("📝 Creating marketing content...")
    
    # STEP 1: Content Writer creates marketing materials
    content_task = f"""
    Create compelling marketing content based on this strategy:
    {sample_strategy}
    
    Generate:
    1. A catchy tagline and value proposition
    2. Key marketing messages for social media
    3. Brief description for app store listing
    4. Elevator pitch for investors (30 seconds)
    
    Make it engaging and action-oriented.
    """
    
    marketing_content = writer(content_task);
    
    print("✅ Marketing content created!")
    print_response(str(marketing_content)[:400] + "...", "Marketing Materials Preview")
    
    print("\n📄 Creating professional report...")
    
    # STEP 2: Report Writer creates formatted report (no file saving)
    report_task = f"""
    Create a comprehensive startup advisory report that includes:
    
    Business Strategy:
    {sample_strategy}
    
    Marketing Materials:
    {marketing_content}
    
    Format this as a professional markdown report with:
    - Executive Summary
    - Business Strategy Overview  
    - Marketing Recommendations
    - Next Steps
    
    Make it well-structured and professional.
    """
    
    report_content = reporter(report_task);
    
    print("✅ Professional report created!")
    
    # Save manually without prompts
    filename = "startup_advisory_report.md"
    save_to_file(report_content, filename)
    
    print_response(str(report_content)[:400] + "...", "Report Content Preview")
    
    return marketing_content, report_content

# Test content creation
marketing_materials, report_status = test_content_creation()

print("\n✅ Step 5 Complete: Agents can now create marketing content and save professional reports!")


In [ ]:
def complete_startup_advisory(startup_description):
    """Complete end-to-end startup advisory workflow"""
    
    print("🚀 COMPLETE STARTUP ADVISORY SYSTEM")
    print("=" * 60)
    print(f"💡 Startup Idea: {startup_description}")
    
    results = {}
    
    # PHASE 1: Market Research  
    print("\n🔍 PHASE 1: Market Research & Analysis")
    print("-" * 50)
    
    research_task = f"""
    Conduct comprehensive market research for: {startup_description}
    
    Research:
    1. Market size and growth potential
    2. Key competitors and their positioning
    3. Target customer segments
    4. Market opportunities and challenges
    
    Be specific and data-driven in your analysis.
    """
    
    if search_client:
        with search_client:
            market_research = analyst(research_task);
    else:
        market_research = analyst(research_task);
    
    results['research'] = market_research
    print("✅ Market research completed")
    
    # PHASE 2: Strategy Development
    print("\n💡 PHASE 2: Business Strategy Development")
    print("-" * 50)
    
    strategy_task = f"""
    Develop a comprehensive business strategy based on this research:
    
    Startup: {startup_description}
    Research: {market_research}
    
    Provide detailed recommendations for:
    1. Value proposition and market positioning
    2. Target customer segments and personas
    3. Competitive differentiation strategy
    4. Go-to-market plan and channels
    5. Revenue model and pricing strategy
    6. Key success metrics and milestones
    """
    
    business_strategy = strategist(strategy_task);
    results['strategy'] = business_strategy
    print("✅ Business strategy developed")
    
    # PHASE 3: Marketing Content
    print("\n✍️ PHASE 3: Marketing Content Creation")
    print("-" * 50)
    
    content_task = f"""
    Create compelling marketing materials for: {startup_description}
    
    Based on strategy: {business_strategy}
    
    Develop:
    1. Brand positioning and tagline
    2. Key value propositions for different audiences
    3. Social media campaign concepts
    4. Investor pitch summary
    5. App store description
    
    Make it engaging and actionable.
    """
    
    marketing_content = writer(content_task);
    results['marketing'] = marketing_content
    print("✅ Marketing content created")
    
    # PHASE 4: Professional Report
    print("\n📄 PHASE 4: Professional Report Generation")
    print("-" * 50)
    
    report_task = f"""
    Create a comprehensive startup advisory report including all our analysis:
    
    Startup: {startup_description}
    Market Research: {market_research}
    Business Strategy: {business_strategy}
    Marketing Content: {marketing_content}
    
    Format as a professional executive report with:
    - Executive Summary
    - Market Analysis
    - Strategic Recommendations
    - Marketing Plan
    - Implementation Roadmap
    - Appendices
    
    Make it well-structured and comprehensive.
    """
    
    final_report = reporter(report_task);
    results['report'] = final_report
    
    # Save the complete report manually (no prompts)
    report_filename = f"complete_startup_advisory_report.md"
    save_to_file(final_report, report_filename)
    print("✅ Professional report generated and saved")
    
    # Summary
    print("\n🎉 STARTUP ADVISORY COMPLETE!")
    print("=" * 60)
    print("📋 Deliverables generated:")
    print("   📊 Market research and competitive analysis")
    print("   💡 Business strategy and recommendations")
    print("   ✍️ Marketing content and campaigns")
    print("   📄 Professional advisory report (saved to file)")
    
    return results

print("🚀 Complete startup advisory system ready!")
print("💡 Ready to analyze any startup idea from market research to final report")


In [ ]:
# Test the complete system with a sample startup
sample_startup = "A subscription service that delivers personalized vitamin packages based on DNA analysis and lifestyle data"

print(f"🧪 Testing complete system with: {sample_startup}")
print("\n⏳ This will take a few moments as each agent does their work...")

# Run the complete workflow
all_results = complete_startup_advisory(sample_startup)

# Display preview of results
print("\n📋 Results Preview:")
print("=" * 50)

for phase, result in all_results.items():
    preview = str(result)[:200] + "..." if len(str(result)) > 200 else str(result)
    print(f"\n{phase.upper()}: {preview}")

print("\n✨ Check the generated report file for complete details!")


In [ ]:
# Try with your own startup idea!
# Uncomment and modify this section to test your own ideas

# your_startup_idea = "Enter your startup idea here..."
# 
# print(f"🚀 Analyzing your startup: {your_startup_idea}")
# your_results = complete_startup_advisory(your_startup_idea)
# 
# print("\n🎉 Your startup analysis is complete!")
# print("📄 Check the generated report for detailed recommendations.")

print("💡 Ready to analyze your startup idea!")
print("✏️ Uncomment the code above and add your startup description to get started.")
